<a href="https://colab.research.google.com/github/HeGanjie/EasyRL_notes/blob/main/EasyRL_note1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

第一章简单总结：

Action、State、Reward 等概念是为了训练方便而做的量化操作

what：增强学习就是让智能体获得收益最大化的算法

how: 通过合理地训练价值函数、策略网络或模型，控制智能体的行为

why 不适用监督学习：是因为环境是可变的，奖励是延迟获得的

需要注意的点：
1. 连续的动作空间，不适合用价值函数
2. 模型相当于智能体的想象力，可以利用它从而缓解训练数据匮乏问题；但是泛化能力更差


下面是用 colab 使用 gym 库 运行 1.7.2 MountainCar-v0 例子并录制视频的 demo:





In [ ]:
# install dependencies needed for recording videos
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.*

In [ ]:
!pip install gym
!pip install numpy

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [ ]:
import gym
# env = gym.make("CartPole-v0")
env = gym.make('MountainCar-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
        env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))

观测空间 = Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
动作空间 = Discrete(3)
观测范围 = [-1.2  -0.07] ~ [0.6  0.07]
动作数 = 3


In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)

In [ ]:
class BespokeAgent:
    def __init__(self, env):
        pass
    
    def decide(self, observation): # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作

    def learn(self, *args): # 学习
        pass
    
agent = BespokeAgent(env)

In [ ]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用 env.close() 语句关闭
            video.capture_frame()
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

In [ ]:
import numpy as np

env.seed(0) # 设置随机数种子,只是为了让结果可以精确复现,一般情况下可删去
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))

# episode_rewards = [play_montecarlo(env, agent) for _ in range(100)]
# print('平均回合奖励 = {}'.format(np.mean(episode_rewards))) # 小车上山环境有一个参考的回合奖励值 −−110，如果连续 100 个回合的平均回合奖励大于 −−110，则认为这个任务被解决了

video.close()
env.close() # 此语句可关闭图形界面

回合奖励 = -105.0


In [ ]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [ ]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

参考文献：
1. [EasyRL 第一章](https://datawhalechina.github.io/easy-rl/#/chapter1/chapter1)
2. [如何在 colab 上查看 gym 渲染结果](https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google)
